In [26]:
import glob
from pathlib import Path  
import numpy as np
import pandas as pd
import geopandas
from shapely.geometry import Point

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [27]:
berkeley_data = pd.read_csv('formatted_allstops_large.csv')
census_tracts = geopandas.read_file('Census Tract Polygons 2010 (With Income+Race+Numstops Data).geojson')

berkeley_data = berkeley_data.rename(columns={'censustract': 'name10'})

/opt/conda/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3369: DtypeWarning: Columns (23,24) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [28]:
# berkeley_data.columns.to_list()

In [29]:
berkeley_data['Date of Stop'] = pd.to_datetime(berkeley_data['Date of Stop'])
berkeley_data['mm'] = berkeley_data['Date of Stop'].dt.month
berkeley_data['yyyy'] = berkeley_data['Date of Stop'].dt.year

berkeley_data = berkeley_data.iloc[:,1:]
berkeley_data.head()

,LEA Record ID,Incident Number,Date of Stop,Time of Stop,Duration of Stop,City,LAT,LONG,Race Perceived Prior To Stop,Perceived Race or Ethnicity,...,Custodial arrest Offense Codes,IncidentNumber,Address,CallType,Car Search,name10,forceused,distancefromcal,mm,yyyy
0,BPD000021033,NaN,2020-10-01,11:14,8.0,Berkeley,37.855313,-122.266571,True,Hispanic/Latino,...,NaN,NaN,NaN,NaN,NaN,4235.0,0,1.2,10,2020
1,BPD000021062,NaN,2020-10-02,15:21,7.0,Berkeley,37.854378,-122.271169,True,Black/African American,...,NaN,NaN,NaN,NaN,NaN,4235.0,1,1.4,10,2020
2,BPD000021048,NaN,2020-10-02,5:04,10.0,Berkeley,37.859469,-122.266941,False,White,...,NaN,NaN,NaN,NaN,NaN,4235.0,0,1.0,10,2020
3,BPD000021083,NaN,2020-10-03,18:57,23.0,Berkeley,37.855689,-122.269154,True,White,...,NaN,NaN,NaN,NaN,NaN,4235.0,0,1.3,10,2020
4,BPD000021126,NaN,2020-10-03,19:56,30.0,Berkeley,37.857611,-122.267249,True,Black/African American,...,PC 459 BURGLARY (F),NaN,NaN,NaN,NaN,4235.0,1,1.1,10,2020


In [30]:
berkeley_data['name10'].unique().size

33

In [31]:
by_census = berkeley_data[['name10']].copy()
by_census.shape
by_census = by_census.groupby('name10').size().to_frame(name='All Time Stop Count')
by_census.shape
by_census['Annual Average Stop Count'] = round((by_census['All Time Stop Count']/8),0)

by_census.head(50)
# by_census = by_census.groupby('name10')
# by_census.shape
# .size().to_frame(name = 'Count').sort_values("Count", ascending = False).head(10).reset_index()
# by_census.rename(columns={'Count':'Total Crimes'})

(61702, 1)

(33, 1)

,All Time Stop Count,Annual Average Stop Count
name10,,
4211.00,111,14.0
4212.00,202,25.0
4213.00,280,35.0
4214.00,160,20.0
4215.00,112,14.0
4216.00,87,11.0
4217.00,677,85.0
4218.00,524,66.0
4219.00,1597,200.0


In [32]:
df = pd.merge(census_tracts, by_census, how='left', left_on='name10', right_on='name10')
df.columns = df.columns.str.replace(' ','_').str.lower()
df.columns = df.columns.str.replace('__','').str.lower()
df = df.drop(columns=['numstops'])
df = df.rename(columns={'all_time_stop_count':'totalnumstops','annual_average_stop_count':'annualstops'})
df['nonwhitecomp'] = 0

for i in range(df.index.size):
    if i == 30:
        continue
    totalpop = float(df.loc[i,'totalpop'].replace(',',''))
    whitepop = float(df.loc[i,'whitepop'].replace(',',''))
    df.loc[i,'nonwhitecomp'] = (totalpop-whitepop)/totalpop

In [41]:
df.head(2)
df.columns.unique

,stroke,stroke-width,stroke-opacity,fill,fill-opacity,mtfcc10,name10,geo,totalpop,intptlat10,...,whitepop,aapop,na_aipop,aisianpop,hawaiian,mixed2,geometry,totalnumstops,annualstops,nonwhitecomp
0,#555555,2,1,#555555,0.5,G5020,4238.0,"Census Tract 4238, Alameda County, California",2925.0,+37.8547587,...,"2,626.0",71.0,0.0,269.0,0.0,154.0,"POLYGON ((-122.24192 37.85202, -122.24180 37.8...",739,92.0,0.102222
1,#555555,2,1,#555555,0.5,G5020,4222.0,"Census Tract 4222, Alameda County, California",3144.0,+37.8728822,...,2125.0,151.0,29.0,405.0,4.0,159.0,"POLYGON ((-122.29365 37.87345, -122.29359 37.8...",3346,418.0,0.324109


<bound method Index.unique of Index(['stroke', 'stroke-width', 'stroke-opacity', 'fill', 'fill-opacity',
       'mtfcc10', 'name10', 'geo', 'totalpop', 'intptlat10', 'awater10',
       'namelsad10', 'id', 'aland10', 'geoid10', 'tractce10', 'intptlon10',
       'medianincome', 'whitepop', 'aapop', 'na_aipop', 'aisianpop',
       'hawaiian', 'mixed2', 'geometry', 'totalnumstops', 'annualstops',
       'nonwhitecomp'],
      dtype='object')>

In [50]:
berkeley_data_for_stata = berkeley_data.copy()

expanded_berkeley_data = pd.merge(berkeley_data_for_stata, df, how='left', left_on='name10', right_on='name10')
expanded_berkeley_data.head(5)
expanded_berkeley_data.shape
# expanded_berkeley_data['nonwhitecomp_y']

,LEA Record ID,Incident Number,Date of Stop,Time of Stop,Duration of Stop,City,LAT,LONG,Race Perceived Prior To Stop,Perceived Race or Ethnicity,...,whitepop,aapop,na_aipop,aisianpop,hawaiian,mixed2,geometry,totalnumstops,annualstops,nonwhitecomp
0,BPD000021033,NaN,2020-10-01,11:14,8.0,Berkeley,37.855313,-122.266571,True,Hispanic/Latino,...,1940.0,585.0,11.0,434.0,6.0,422.0,"POLYGON ((-122.26471 37.86157, -122.26474 37.8...",3486,436.0,0.377806
1,BPD000021062,NaN,2020-10-02,15:21,7.0,Berkeley,37.854378,-122.271169,True,Black/African American,...,1940.0,585.0,11.0,434.0,6.0,422.0,"POLYGON ((-122.26471 37.86157, -122.26474 37.8...",3486,436.0,0.377806
2,BPD000021048,NaN,2020-10-02,5:04,10.0,Berkeley,37.859469,-122.266941,False,White,...,1940.0,585.0,11.0,434.0,6.0,422.0,"POLYGON ((-122.26471 37.86157, -122.26474 37.8...",3486,436.0,0.377806
3,BPD000021083,NaN,2020-10-03,18:57,23.0,Berkeley,37.855689,-122.269154,True,White,...,1940.0,585.0,11.0,434.0,6.0,422.0,"POLYGON ((-122.26471 37.86157, -122.26474 37.8...",3486,436.0,0.377806
4,BPD000021126,NaN,2020-10-03,19:56,30.0,Berkeley,37.857611,-122.267249,True,Black/African American,...,1940.0,585.0,11.0,434.0,6.0,422.0,"POLYGON ((-122.26471 37.86157, -122.26474 37.8...",3486,436.0,0.377806


(61702, 82)

In [51]:
expanded_berkeley_data = expanded_berkeley_data.rename(columns={'totalnumstops':'area_totalstops',
                                                                'annualstops':'area_annualstops',
                                                                'totalpop':'area_totalpop',
                                                                'medianincome':'area_medianincome',
                                                                'name10':'censustract'})
expanded_berkeley_data = expanded_berkeley_data.drop(columns=['stroke', 'stroke-width', 'stroke-opacity', 'fill',
                                                              'fill-opacity', 'mtfcc10', 'geo','intptlat10', 'awater10',
                                                              'namelsad10', 'id', 'aland10', 'geoid10', 'tractce10', 'intptlon10','aapop', 'na_aipop', 'aisianpop',
                                                              'hawaiian', 'whitepop', 'mixed2', 'geometry'])
expanded_berkeley_data.columns.unique

<bound method Index.unique of Index(['LEA Record ID', 'Incident Number', 'Date of Stop', 'Time of Stop',
       'Duration of Stop', 'City', 'LAT', 'LONG',
       'Race Perceived Prior To Stop', 'Perceived Race or Ethnicity',
       'Perceived Gender', 'Perceived Age', 'Reason for Stop',
       'Reason for Stop Narrative', 'Result of Stop', 'Person Number',
       'Is Stop Made in Response to Call for Service',
       'Information Based Stop', 'Type Of Stop', 'Officer Type of Assignment',
       'Location', 'Is Location a K12 Public School',
       'If K12 School Is Stop of a Student', 'School Name',
       'Education Code Section', 'Education Code Subdivision',
       'Perceived Gender Nonconforming', 'Is LGBT',
       'Person had Limited or No English Fluency',
       'Perceived or Known Disability', 'City of Residence',
       'Traffic Violation Type', 'Traffic Violation Offense Codes',
       'Suspicion Offense Code', 'Suspicion SubType', 'Actions Taken',
       'Basis for Search', 

In [52]:
filepath = Path('formatted_allstops_large(expanded).csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
expanded_berkeley_data.to_csv(filepath, na_rep='')

In [ ]:
# df.to_file("Census Tract Polygons 2010 (With Income+Race+Numstops Data)(Expanded).geojson", driver='GeoJSON')
